# HR Analytics

<img src = 'https://datahack-prod.s3.ap-south-1.amazonaws.com/__sized__/contest_cover/hr_1920x480_s5WuoZs-thumbnail-1200x1200-90.jpg'>

Practice Problem: https://datahack.analyticsvidhya.com/contest/wns-analytics-hackathon-2018-1/

## HR Analytics

HR analytics is revolutionising the way human resources departments operate, leading to higher efficiency and better results overall. Human resources has been using analytics for years. However, the collection, processing and analysis of data has been largely manual, and given the nature of human resources dynamics and HR KPIs, the approach has been constraining HR. Therefore, it is surprising that HR departments woke up to the utility of machine learning so late in the game. Here is an opportunity to try predictive analytics in identifying the employees most likely to get promoted.

## Problem Statement

Your client is a large MNC and they have 9 broad verticals across the organisation. One of the problem your client is facing is around identifying the right people for promotion *(only for manager position and below)* and prepare them in time. Currently the process, they are following is:

* They first identify a set of employees based on recommendations/ past performance
* Selected employees go through the separate training and evaluation program for each vertical. These programs are based on the required skill of each vertical
* At the end of the program, based on various factors such as training performance, KPI completion (only employees with KPIs completed greater than 60% are considered) etc., employee gets promotion

For above mentioned process, the final promotions are only announced after the evaluation and this leads to delay in transition to their new roles. Hence, company needs your help in identifying the eligible candidates at a particular checkpoint so that they can expedite the entire promotion cycle. 

<img src = 'https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2018/09/wns_hack_im_1.jpg'>

They have provided multiple attributes around Employee's past and current performance along with demographics. Now, The task is to predict whether a potential promotee at checkpoint in the test set will be promoted or not after the evaluation process.

## Evaluation Metric

The evaluation metric for this competition is F1 Score.

## Public and Private Split

Test data is further randomly divided into Public (40%) and Private (60%) data.

Your initial responses will be checked and scored on the Public data.
The final rankings would be based on your private score which will be published once the competition is over.

## Entorno

In [1]:
import sys
sys.version

'3.6.12 |Anaconda, Inc.| (default, Sep  8 2020, 17:50:39) \n[GCC Clang 10.0.0 ]'

In [2]:
!conda info --envs

# conda environments:
#
micromaster              /Users/manuel/.conda/envs/micromaster
                         /Users/manuel/.julia/conda/3
base                  *  /Users/manuel/opt/anaconda3
belcorp                  /Users/manuel/opt/anaconda3/envs/belcorp
courseragcp              /Users/manuel/opt/anaconda3/envs/courseragcp
iapucp                   /Users/manuel/opt/anaconda3/envs/iapucp
mitxpro                  /Users/manuel/opt/anaconda3/envs/mitxpro
style-transfer           /Users/manuel/opt/anaconda3/envs/style-transfer
taller-dmc               /Users/manuel/opt/anaconda3/envs/taller-dmc
udacity                  /Users/manuel/opt/anaconda3/envs/udacity



## Paquetes

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm, tqdm_notebook
from pathlib import Path
import random
import warnings
import pickle

warnings.filterwarnings('ignore')


seed = 2020
random.seed(seed)

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 400)
sns.set()

DATA = Path('../../data') 
RAW  = DATA/'raw'
PROCESSED = DATA/'processed'
SUBMISSIONS = DATA/'submissions'    
MODEL_RESULTS = DATA/'models'

MODEL = Path('../../model') 

In [4]:
pd.__version__

'1.1.3'

In [5]:
np.__version__

'1.19.1'

In [6]:
sklearn.__version__

'0.23.2'

In [7]:
id_columns = 'employee_id'
target = 'is_promoted'

In [8]:
model_label = 'xgboost_20'

## Elección del modelo

In [9]:
os.listdir(f'{MODEL_RESULTS}')

['xgboost_baseline.csv']

In [10]:
df_results = pd.read_csv(f'{MODEL_RESULTS}/xgboost_baseline.csv')
df_results

,preproc_label,model_label,método,parámetros,columnas_out,auc_train,auc_val,threshold,f1_train,f1_val
0,preprocess_v1_over50,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 0, 'max_depth'...",NaN,0.925740,0.912398,0.519916,0.857260,0.387935
1,preprocess_v1_smote20,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 0, 'max_depth'...",NaN,0.963750,0.902189,0.363386,0.809268,0.303494
2,preprocess_v1_smote50,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 0, 'max_depth'...",NaN,0.994993,0.903298,0.397081,0.970218,0.316904
3,preprocess_v1_smoteTomek20,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 0, 'max_depth'...",NaN,0.984355,0.896496,0.335935,0.873732,0.352239
4,preprocess_v1_smoteTomek50,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 0, 'max_depth'...",NaN,0.993239,0.902251,0.425842,0.967290,0.307300
...,...,...,...,...,...,...,...,...,...,...
499,preprocess_v2_smote50,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 5, 'max_depth'...",NaN,0.993078,0.909270,0.449836,0.966119,0.519499
500,preprocess_v2_smoteTomek20,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 5, 'max_depth'...",NaN,0.972369,0.909576,0.357857,0.834015,0.543329
501,preprocess_v2_smoteTomek50,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 5, 'max_depth'...",NaN,0.993535,0.909469,0.429405,0.966711,0.522215
502,preprocess_v2,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 5, 'max_depth'...",NaN,0.917067,0.909057,0.256593,0.542299,0.531928


In [11]:
df_results = df_results.sort_values(by = 'f1_val', ascending = False).head()
df_results

,preproc_label,model_label,método,parámetros,columnas_out,auc_train,auc_val,threshold,f1_train,f1_val
19,preprocess_v1,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 0, 'max_depth'...",NaN,0.936164,0.912828,0.264434,0.578571,0.555488
290,preprocess_v2_smoteTomek20,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 2.5, 'max_dept...",NaN,0.977697,0.914479,0.355524,0.853583,0.548159
67,preprocess_v2_smoteTomek50,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 0, 'max_depth'...",NaN,0.995018,0.912623,0.398115,0.970562,0.548117
9,preprocess_v2_smote50,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 0, 'max_depth'...",NaN,0.994278,0.912402,0.402771,0.968889,0.546985
245,preprocess_v2_over50,xgboost_baseline,fit,"{'eval_metric': 'auc', 'gamma': 2.5, 'max_dept...",NaN,0.993614,0.912404,0.367706,0.968155,0.545322


In [12]:
preproc_label = df_results.head(1)['preproc_label'].values[0]
params = df_results.head(1)['parámetros'].values[0]

In [13]:
preproc_label

'preprocess_v1'

In [14]:
params

"{'eval_metric': 'auc', 'gamma': 0, 'max_depth': 3, 'min_child_weight': 10, 'objective': 'reg:logistic', 'seed': 2020}"

## Bases

In [15]:
df_train = pd.read_csv(f'{PROCESSED}/{preproc_label}_train.csv', compression = 'zip')
df_val = pd.read_csv(f'{PROCESSED}/{preproc_label}_val.csv', compression = 'zip')
df_test = pd.read_csv(f'{PROCESSED}/{preproc_label}_test.csv', compression = 'zip')

In [16]:
df_train.shape, df_val.shape, df_test.shape

((43846, 61), (10962, 61), (23490, 61))

In [17]:
X_train, y_train = df_train.drop(target, axis = 1), df_train[target]
X_val, y_val = df_val.drop(target, axis = 1), df_val[target]

ids_test = df_test[id_columns]
X_test = df_test.drop(id_columns, axis = 1)

## Training

In [18]:
import xgboost as xgb
from sklearn.metrics import roc_auc_score, f1_score, precision_recall_curve

In [19]:
dtrain =  xgb.DMatrix(data=X_train, label = y_train)
dval  = xgb.DMatrix(data=X_val,   label = y_val)

watch_list  = [(dtrain,'train'),(dval,'val')]

xgb_fit = xgb.train(params = eval(params), dtrain = dtrain, 
                    num_boost_round = 1000, early_stopping_rounds = 20, 
                    evals = watch_list, verbose_eval=50)

[0]	train-auc:0.74958	val-auc:0.75430
Multiple eval metrics have been passed: 'val-auc' will be used for early stopping.

Will train until val-auc hasn't improved in 20 rounds.
[50]	train-auc:0.91216	val-auc:0.90858
[100]	train-auc:0.92173	val-auc:0.91223
Stopping. Best iteration:
[93]	train-auc:0.92097	val-auc:0.91229



In [20]:
probs_train = xgb_fit.predict(dtrain, ntree_limit = xgb_fit.best_iteration)
probs_val = xgb_fit.predict(dval, ntree_limit = xgb_fit.best_iteration)

auc_train = roc_auc_score(y_train, probs_train)
auc_val = roc_auc_score(y_val, probs_val)

#best threshold
prec, recall, threshold = precision_recall_curve(y_train, probs_train)
prec_recall = pd.DataFrame({'prec': prec[:-1], 'recall': recall[:-1], 'threshold': threshold})
prec_recall['f1'] = 2*prec_recall['prec']*prec_recall['recall'] / (prec_recall['prec'] + prec_recall['recall'])
prec_recall = prec_recall.sort_values(by = 'f1', ascending = False).head(1)

#f1 scores
best_threshold = prec_recall['threshold'].values[0]
f1_train = prec_recall['f1'].values[0]

labels_val = np.where(probs_val >= best_threshold, 1, 0)
f1_val = f1_score(y_val, labels_val)

print(f'auc_train: {auc_train:.6f} \tauc_val: {auc_val:.6f} \tf1_train: {f1_train:.6f} \tf1_val: {f1_val:.6f}')

auc_train: 0.920889 	auc_val: 0.912261 	f1_train: 0.536326 	f1_val: 0.528829


## Training with complete train + validation

In [21]:
df_train = pd.concat([df_train, df_val])
df_train.shape

(54808, 61)

In [22]:
X_train, y_train = df_train.drop(target, axis = 1), df_train[target]
dtrain =  xgb.DMatrix(data=X_train, label = y_train)

In [23]:
xgb_fit = xgb.train(params = eval(params), dtrain = dtrain, 
                    num_boost_round = xgb_fit.best_iteration, verbose_eval=50)

In [24]:
probs_train = xgb_fit.predict(dtrain)

auc_train = roc_auc_score(y_train, probs_train)

#best threshold
prec, recall, threshold = precision_recall_curve(y_train, probs_train)
prec_recall = pd.DataFrame({'prec': prec[:-1], 'recall': recall[:-1], 'threshold': threshold})
prec_recall['f1'] = 2*prec_recall['prec']*prec_recall['recall'] / (prec_recall['prec'] + prec_recall['recall'])
prec_recall = prec_recall.sort_values(by = 'f1', ascending = False).head(1)

#f1 scores
best_threshold = prec_recall['threshold'].values[0]
f1_train = prec_recall['f1'].values[0]

print(f'auc_train: {auc_train:.6f} \tf1_train: {f1_train:.6f}')

auc_train: 0.919474 	f1_train: 0.534151


In [25]:
dtest =  xgb.DMatrix(data = X_test)

In [26]:
probs_test = xgb_fit.predict(dtest)
labels_test = np.where(probs_test >= best_threshold, 1, 0)

In [27]:
df_test[id_columns] = ids_test
df_test[target] = labels_test
df_test.get([id_columns, target])

,employee_id,is_promoted
0,8724,0
1,74430,0
2,72255,0
3,38562,0
4,64486,0
...,...,...
23485,53478,0
23486,25600,0
23487,45409,0
23488,1186,0


In [28]:
df_test.get([id_columns, target]).to_csv(f'{SUBMISSIONS}/{model_label}_test.csv', index = False)